# Identifying the topics based on questions asked by Amazon customers

### What is topic modelling?
It is an unsupervised approach used for finding and observing the bunch of words (called Topics) in large clusters of texts.

### Algorithm used:
There are many algorithms available for Topic Modelling such as LSA (Letent Semantic Analysis), NMF (Non-Matrix Factorization) and LDA (Latent Dirichlet Algorithm). I'm using LDA as it provides more accurate results and scales well for large text corpuses.

### A brief intro to LDA algorithm:
LDA is a matrix factorization technique which assumes sentences are made up of mixture of words and tries to figure out what words would create those sentences in the first place. It is based on probabilistic graphical modelling. The algorithm works as follows:<br>
1. In the initialization stage, each word is assigned to a random topic.
2. Iteratively, the algorithm goes through each word and reassigns the word to a topic taking into consideration:<br>
    What’s the probability of the word belonging to a topic?<br>
    What’s the probability of the document to be generated by a topic?
[Source]('https://nlpforhackers.io/topic-modeling/')
### Data Source:
http://jmcauley.ucsd.edu/data/amazon/qa/. Click on the Automotive category file to download the json file having the customer queries.

#### Step 1: Import the required libraries

In [13]:
#Hold the data in a table
import pandas as pd
# import numpy as np

#Read the zip files
import gzip

#Ignore warnings
import warnings
warnings.filterwarnings("ignore")

#Data Preprocessing
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
# np.random.seed(2018)
# import nltk
# nltk.download('wordnet')

#### Step 2: Import the data into dataframe

In [4]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

data = getDF('qa_Automotive.json.gz')

#### Step 3: Extract the text column and assign it to an index

In [5]:
def prepare_data(file_name):
    data_text = data[['question']]
    data_text['index'] = data_text.index
    return data_text
documents=prepare_data(data)

Overview of the file

In [6]:
documents.head()

,question,index
0,What is the most useful length to get?,0
1,Are these cables made of copper or aluminum?,1
2,I bought the Red Extra Heavy Duty. Is that too...,2
3,"Hi, Being 20ft 4gauge how heavy is this?",3
4,Do these cables come with a bag?,4


#### Step 4: Reducing the words to its common  base form
Now, we need to reduce the words which are in its plural or derivational forms into its common base form. This will help us collect the similar words from all the corpuses. Now the question is how do we do that? Here, stemming and lemmatization comes to the rescue.<br><br>
**What is Stemming?** <br>
The process of chopping off the ends of words, often includes the removal of derivational fixes. E.g. cats -> cat<br>
There are two types of stemmer: SnowballStemmer and PorterStemmer. We will be using SnowballStemmer as it gives a more meaningful result compared to PorterStemmer().<br>

**What is Lemmatization?** <br>
The process of removal of inflectional endings only and returning the base or dictionary form of a word, which is known as lemma. E.g. running -> run.<br>

In [9]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
stemmer = SnowballStemmer('english')

Next, we need to remove the stopwords and words whose length is less than or equal to three as they won't add much value to our model. We will implement it using gensim library as it already has a tokenization function and built-in library of stopwords.

In [11]:
def remove_stopwords_and_shortwords(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

Let's look at an example of how it works:

In [14]:
sample = documents[documents['index'] == len(documents)-1].values[0][0]

print('original document: ')
words = []
for word in sample.split(' '):
    words.append(word)
print(words)
print('\n\n Tokenized and lemmatized document: ')
print(remove_stopwords_and_shortwords(sample))

original document: 
['Travis', 'Hellmer', 'said,', '"DO', 'NOT', 'USE', 'on', 'the', 'handgrips', 'or', 'passenger', 'seat!"', 'Why', 'not?']


 Tokenized and lemmatized document: 
['travi', 'hellmer', 'say', 'handgrip', 'passeng', 'seat']


Now, we will apply the same on our dataset.

In [15]:
processed_docs = documents['question'].map(remove_stopwords_and_shortwords)

Overview of the processed docs

In [16]:
processed_docs[:10]

0                      [use, length]
1           [cabl, copper, aluminum]
2    [buy, extra, heavi, duti, size]
3                      [gaug, heavi]
4                       [cabl, come]
5        [wire, pair, surpris, wire]
6                       [amp, handl]
7              [cabl, boost, school]
8                      [use, length]
9           [cabl, copper, aluminum]
Name: question, dtype: object

#### Step 5: Build a dictionary of words present in the dataset

In [17]:
dictionary = gensim.corpora.Dictionary(processed_docs)
print('There are total '+ str(len(dictionary))+' words in the dictionary.')

There are total 14469 words in the dictionary.


An overview of the dictionary:

In [18]:
count = 0
for index, word in dictionary.iteritems():
    print(index, word)
    count += 1
    if count > 10:
        break

0 length
1 use
2 aluminum
3 cabl
4 copper
5 buy
6 duti
7 extra
8 heavi
9 size
10 gaug


#### Step 6: Filter the dictionary based on frequency of words. 
Here, I have ignored the words which have appeared in less than 4 queries and the ones who are present in at most 50% of the documents.

In [19]:
dictionary.filter_extremes(no_below=4, no_above=0.5)
print('No. of words in the dictionary after filtering: '+ str(len(dictionary)))

No. of words in the dictionary after filtering: 4705


#### Step 7: Create a list having the index value and the frequency of the word for every question in the dataset.

In [20]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[:5]

[[(0, 1), (1, 1)],
 [(2, 1), (3, 1), (4, 1)],
 [(5, 1), (6, 1), (7, 1), (8, 1), (9, 1)],
 [(8, 1), (10, 1)],
 [(3, 1), (11, 1)]]

A sample:

In [21]:
bow_doc_sample = bow_corpus[0]

for i in range(len(bow_doc_sample)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_sample[i][0], 
                                                     dictionary[bow_doc_sample[i][0]], 
                                                     bow_doc_sample[i][1]))

Word 0 ("length") appears 1 time.
Word 1 ("use") appears 1 time.
